In [1]:
import pandas as pd

# 1. Re-arrange data from RASPA calculation

In [7]:
df_raspa_no_tailcorrection = pd.read_csv('no_tailcorrection_data.csv')
df_raspa_no_tailcorrection_bp = pd.read_csv('no_tailcorrection_data_blocked_pockets.csv')

In [8]:
df_raspa_tailcorrection = pd.read_csv('tailcorrection_data.csv')
df_raspa_tailcorrection_bp = pd.read_csv('tailcorrection_data_blocked_pockets.csv')

In [3]:
blocked_pockets = [
    "15072N2",
    "16083N2",
    "17120N2",
    "18091N2",
    "AFT",
    "BENXUP",
    "EMT",
    "GOO",
    "PAU",
    "PIHNUQ",
    "SBMOF-1",
    "TSC",
    "TUDJIM",
    "UEI",
    "XOJWID",
    "YUJNIB",
    "ZIF-4",
]

non_permeable = [
    "13030N2",
    "17120N2",
    "ZIF-4",
    "SBMOF-1",
    "ZIF-8",
    "PAU",
    "AFT",
    "TSC",
    "GOO",
    "UEI",
    "XOJWID",
    "TUDJIM",
    "PIHNUQ",
    "BENXUP",
    "YUJNIB",
]

used_block = list(set(blocked_pockets) - set(non_permeable))

In [4]:
used_block

['EMT', '15072N2', '16083N2', '18091N2']

In [9]:
df_raspa_no_tailcorrection_selection = df_raspa_no_tailcorrection[~df_raspa_no_tailcorrection['name'].isin(used_block)]

In [11]:
df_raspa_no_tailcorrection_bp_selection = df_raspa_no_tailcorrection_bp[df_raspa_no_tailcorrection_bp['name'].isin(
used_block)]

In [12]:
df_raspa_tailcorrection_selection = df_raspa_tailcorrection[~df_raspa_tailcorrection['name'].isin(used_block)]

In [13]:
df_raspa_tailcorrection_bp_selection = df_raspa_tailcorrection_bp[df_raspa_tailcorrection_bp['name'].isin(
used_block)]

In [14]:
df_raspa_tailcorrection = pd.concat([df_raspa_tailcorrection_selection, df_raspa_tailcorrection_bp_selection])

In [15]:
df_raspa_no_tailcorrection = pd.concat([df_raspa_no_tailcorrection_selection, df_raspa_no_tailcorrection_bp_selection])

In [56]:
df_raspa_tailcorrection.to_csv('tc_data_materials_cloud.csv', index=False)
df_raspa_no_tailcorrection.to_csv('no_tc_data_materials_cloud.csv', index=False)

In [16]:
# Make further subsets based on cutoff distance
df_raspa_tailcorrection_12 = df_raspa_tailcorrection[df_raspa_tailcorrection['cutoff'] == 12]
df_raspa_tailcorrection_20 = df_raspa_tailcorrection[df_raspa_tailcorrection['cutoff'] == 20]

df_raspa_no_tailcorrection_12 = df_raspa_no_tailcorrection[df_raspa_no_tailcorrection['cutoff'] == 12]
df_raspa_no_tailcorrection_20 = df_raspa_no_tailcorrection[df_raspa_no_tailcorrection['cutoff'] == 20]

In [17]:
df_raspa_no_tailcorrection_12.columns.values

array(['Unnamed: 0', 'adsorption_energy_widom_average',
       'adsorption_energy_widom_dev', 'cell_volume_average',
       'cell_volume_dev', 'cell_volume_units', 'cf',
       'chemical_potential_average', 'chemical_potential_dev', 'cutoff',
       'ff', 'framework_density', 'framework_density_units',
       'gcmc_high_p_host_ads_total_energy_average',
       'gcmc_high_p_host_ads_total_energy_dev',
       'gcmc_high_p_host_ads_vdw_energy_average',
       'gcmc_high_p_host_ads_vdw_energy_dev',
       'gcmc_high_p_tail_correction_energy_average',
       'gcmc_high_p_tail_correction_energy_dev',
       'gcmc_high_p_total_energy_average', 'gcmc_high_p_total_energy_dev',
       'gcmc_low_p_host_ads_total_energy_average',
       'gcmc_low_p_host_ads_total_energy_dev',
       'gcmc_low_p_host_ads_vdw_energy_average',
       'gcmc_low_p_host_ads_vdw_energy_dev',
       'gcmc_low_p_tail_correction_energy_average',
       'gcmc_low_p_tail_correction_energy_dev',
       'gcmc_low_p_total_energy

In [18]:
# now define the relevant columns to make the next steps a bit cleaner/easier 

relevant_raspa_columns = [
    'name', 
    'henry_coefficient_widom_average',
    'henry_coefficient_widom_dev',
    'loading_absolute_average_high_p', 
    'loading_absolute_average_low_p',
    'loading_absolute_average_medium_p',
    'loading_absolute_dev_high_p', 
    'loading_absolute_dev_low_p',
    'loading_absolute_dev_medium_p'
]

In [19]:
df_raspa_tailcorrection_12_selected = df_raspa_tailcorrection_12[relevant_raspa_columns]
df_raspa_tailcorrection_20_selected = df_raspa_tailcorrection_20[relevant_raspa_columns]

df_raspa_no_tailcorrection_12_selected = df_raspa_no_tailcorrection_12[relevant_raspa_columns]
df_raspa_no_tailcorrection_20_selected = df_raspa_no_tailcorrection_20[relevant_raspa_columns]

In [20]:
rename_dict_base = {
    'henry_coefficient_widom_average': 'kh',
    'henry_coefficient_widom_dev': 'kh_error',
    'loading_absolute_average_high_p': 'loading_65', 
    'loading_absolute_average_low_p': 'loading_5_8', 
    'loading_absolute_average_medium_p': 'loading_35', 
    'loading_absolute_dev_high_p': 'loading_65_error', 
    'loading_absolute_dev_low_p': 'loading_5_8_error',
    'loading_absolute_dev_medium_p': 'loading_35_error'
}

def rename_columns(df, rename_dict_base=rename_dict_base, rename_extension='_20_no_tc'):
    _rename_keys = rename_dict_base.keys()
    _rename_values = [n + rename_extension for n in rename_dict_base.values()]
    rename_dict = dict(zip(_rename_keys, _rename_values))
    df['dc' + rename_extension] = df['loading_absolute_average_high_p'] - df['loading_absolute_average_low_p']
    df = df.rename(columns=rename_dict)
    return df 

In [21]:
df_raspa_no_tailcorrection_20_selected = rename_columns(df_raspa_no_tailcorrection_20_selected, 
                                                        rename_extension='_20_no_tc')

df_raspa_no_tailcorrection_12_selected = rename_columns(df_raspa_no_tailcorrection_12_selected, 
                                                        rename_extension='_12_no_tc')


df_raspa_tailcorrection_20_selected = rename_columns(df_raspa_tailcorrection_20_selected, 
                                                        rename_extension='_20_tc')

df_raspa_tailcorrection_12_selected = rename_columns(df_raspa_tailcorrection_12_selected, 
                                                        rename_extension='_12_tc')

/home/kevin/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [22]:
df_raspa_tailcorrection_12_selected.head()

name      kh_12_tc  kh_error_12_tc  loading_65_12_tc  \
6   13030N2  3.639850e-56    1.522980e-56          0.000000   
14  15030N2  3.229330e-06    1.283860e-08        146.435555   
44  16411C2  3.959350e-06    7.080830e-08        124.493651   
63  17030N2  3.701710e-06    2.579250e-08        133.139076   
71  17040N2  4.858910e-06    3.738280e-08        158.922723   

    loading_5_8_12_tc  loading_35_12_tc  loading_65_error_12_tc  \
6            0.000000          0.000000                0.000000   
14          22.500470        102.764021                1.056262   
44          32.481784         98.128778                0.984927   
63          18.493741         86.946663                1.121862   
71          40.414400        130.758574                0.792140   

    loading_5_8_error_12_tc  loading_35_error_12_tc    dc_12_tc  
6                  0.000000                0.000000    0.000000  
14                 0.484027                0.836320  123.935085  
44                 1.554961                1.614323   92.011867  
63                 0.508171                2.136057  114.645335  
71                 0.981810                0.726374  118.508323

Now, we can merge all the dataframes back

In [23]:
__df_raspa_merged = df_raspa_no_tailcorrection_20_selected.merge(df_raspa_tailcorrection_20_selected, 
                                                               left_on='name', right_on='name')

_df_raspa_merged = df_raspa_no_tailcorrection_12_selected.merge(df_raspa_tailcorrection_12_selected, 
                                                               left_on='name', right_on='name')

df_raspa_merged = _df_raspa_merged.merge(__df_raspa_merged, left_on='name', right_on='name')

In [24]:
df_raspa_merged.head()

name   kh_12_no_tc  kh_error_12_no_tc  loading_65_12_no_tc  \
0  13030N2  2.056090e-56       1.612610e-56             0.000000   
1  15030N2  2.701440e-06       1.364100e-08           128.065247   
2  16411C2  3.104940e-06       5.944330e-08           110.779208   
3  17030N2  3.268900e-06       1.313540e-08           117.527150   
4  17040N2  3.848890e-06       2.381050e-08           144.987710   

   loading_5_8_12_no_tc  loading_35_12_no_tc  loading_65_error_12_no_tc  \
0              0.000000             0.000000                   0.000000   
1             19.244052            87.614821                   1.721082   
2             26.670975            85.559362                   1.801933   
3             16.482524            77.417326                   0.646303   
4             32.470629           115.025790                   1.280339   

   loading_5_8_error_12_no_tc  loading_35_error_12_no_tc  dc_12_no_tc  ...  \
0                    0.000000                   0.000000     0.000000  ...   
1                    0.471007                   1.910243   108.821195  ...   
2                    0.908552                   1.466130    84.108233  ...   
3                    0.707367                   1.430119   101.044625  ...   
4                    1.134927                   1.713823   112.517081  ...   

   dc_20_no_tc      kh_20_tc  kh_error_20_tc  loading_65_20_tc  \
0     0.000000  3.810330e-56    2.336630e-56          0.000000   
1   122.330977  3.244570e-06    3.222470e-08        147.824098   
2    90.087869  3.937200e-06    7.655750e-08        124.480634   
3   111.328564  3.717470e-06    4.035070e-08        133.218227   
4   117.457699  4.915520e-06    3.443250e-08        160.090208   

   loading_5_8_20_tc  loading_35_20_tc  loading_65_error_20_tc  \
0           0.000000          0.000000                0.000000   
1          22.852577        103.779865                1.328681   
2          33.049305         99.871166                0.516457   
3          18.706251         87.725781                0.567143   
4          41.632250        131.310573                0.703769   

   loading_5_8_error_20_tc  loading_35_error_20_tc    dc_20_tc  
0                 0.000000                0.000000    0.000000  
1                 0.322358                0.371183  124.971520  
2                 0.339228                0.371758   91.431330  
3                 0.351579                0.471500  114.511976  
4                 0.421797                0.534575  118.457958  

[5 rows x 37 columns]

## 2. import the zeo++

In [25]:
df_zeopp_famous_mofs = pd.read_csv('famous_mof_pore_properties.csv')
df_zeopp_mofs = pd.read_csv('mof_pore_properties.csv')
df_zeopp_cofs = pd.read_csv('cof_pore_properties.csv')
df_zeopp_zeolites = pd.read_csv('zeolite_pore_properties.csv')

In [26]:
df_zeopp_mofs[df_zeopp_mofs['POAV_Volume_fraction'] < 1e-4]

Unnamed: 0  ASA_m^2/g   Density  Largest_free_sphere  \
0             0        0.0  1.373360              3.32333   
1             1        0.0  1.780400              2.49950   
2             2        0.0  1.200490              3.69512   
5             5        0.0  2.333490              1.93931   
6             6        0.0  1.488240              2.24353   
7             7        0.0  1.782030              3.68719   
11           11        0.0  1.607070              2.99481   
13           13        0.0  2.299360              2.40540   
15           15        0.0  1.016160              3.57635   
16           16        0.0  1.231240              3.10798   
18           18        0.0  1.690940              2.54408   
19           19        0.0  1.343560              1.78066   
21           21        0.0  1.590200              2.70649   
22           22        0.0  2.286650              3.25414   
23           23        0.0  2.035850              2.73158   
26           26        0.0  1.790230              3.70729   
28           28        0.0  1.442790              3.70108   
30           30        0.0  1.954110              2.81029   
37           37        0.0  1.329640              3.38506   
40           40        0.0  1.386180              2.86111   
41           41        0.0  1.255510              3.53499   
46           46        0.0  2.118890              2.82313   
48           48        0.0  1.784090              3.70002   
49           49        0.0  1.566200              2.82955   
54           54        0.0  1.248850              2.20253   
55           55        0.0  1.324710              3.11042   
59           59        0.0  2.093880              2.07526   
60           60        0.0  1.157940              2.76885   
62           62        0.0  1.408620              3.31204   
64           64        0.0  1.359320              2.48784   
..          ...        ...       ...                  ...   
762         762        0.0  1.386970              2.64832   
763         763        0.0  1.169860              3.28556   
765         765        0.0  1.913720              3.30264   
767         767        0.0  1.867000              3.24288   
771         771        0.0  3.162480              2.70473   
772         772        0.0  2.022480              2.41122   
776         776        0.0  0.901586              3.45968   
777         777        0.0  1.167660              3.20372   
780         780        0.0  1.307080              3.01829   
786         786        0.0  1.717950              2.10444   
793         793        0.0  1.606480              2.95054   
794         794        0.0  1.395420              2.24475   
796         796        0.0  1.849400              2.48074   
797         797        0.0  0.908061              3.43064   
798         798        0.0  1.854530              2.97961   
799         799        0.0  0.977571              3.51349   
801         801        0.0  1.155840              3.46929   
808         808        0.0  1.510790              2.16432   
809         809        0.0  1.609420              2.18377   
811         811        0.0  0.907479              3.42544   
812         812        0.0  1.386330              2.53399   
813         813        0.0  1.363360              2.97635   
818         818        0.0  1.287370              2.29813   
820         820        0.0  1.736150              2.60452   
821         821        0.0  1.366110              3.58255   
823         823        0.0  0.978863              3.48369   
825         825        0.0  1.688830              2.36049   
826         826        0.0  1.359000              3.53705   
832         832        0.0  1.551820              2.94071   
833         833        0.0  1.314980              2.44922   

     Largest_included_free_sphere  Largest_included_sphere   NASA_m^2/g  \
0                         4.22208                  4.22448    84.483400   
1                         3.80274                  3.81032     2.072660 

In [27]:
df_zeopp_zeolites.head()

Unnamed: 0  ASA_m^2/g  Density  Largest_free_sphere  \
0           0    101.053  1.87081              3.79966   
1           1      0.000  1.66165              2.13224   
2           2    564.112  1.60531              3.73943   
3           3      0.000  1.75452              3.10046   
4           4    903.390  1.45923              6.94395   

   Largest_included_free_sphere  Largest_included_sphere  NASA_m^2/g  \
0                       4.73354                  5.60181     81.0896   
1                       5.80124                  5.82045    300.6230   
2                       7.63477                  7.63706      0.0000   
3                       3.61163                  3.61163      0.0000   
4                      10.39668                 10.39668      0.0000   

   Number_of_channels  Number_of_pockets  POAV_Volume_fraction  \
0                   2                  2               0.12090   
1                   0                  2               0.00000   
2                   2                  0               0.36644   
3                   0                  0               0.00000   
4                   1                  0               0.45947   

   Pocket_surface_area_A^2     name  
0                  24.3478  PCR.cif  
1                  18.0302  SOD.cif  
2                   0.0000  RTH.cif  
3                   0.0000  ABW.cif  
4                   0.0000  SBT.cif

In [28]:
rename_dict_zeopp = {
    'ASA_m^2/g': 'accessible_surface_area',
    'Density': 'density',
    'Largest_free_sphere': 'largest_free_sphere',
    'Largest_included_sphere': 'largest_included_sphere',
    'Largest_included_free_sphere': 'largest_included_free_sphere',
    'POAV_Volume_fraction': 'poav',
    'Pocket_surface_area_A^2': 'pocket_surface_area'
}

In [29]:
def rename_zeopp_df(df, rename_dict=rename_dict_zeopp):
    df['name'] = [n.strip('.cif') for n in df['name'].values]
    df = df.rename(columns=rename_dict)
    relevant_columns = rename_dict.values() + ['name']
    return df[relevant_columns]

In [30]:
df_zeopp_cofs_rewritten =  rename_zeopp_df(df_zeopp_cofs)
df_zeopp_mofs_rewritten =  rename_zeopp_df(df_zeopp_mofs)
df_zeopp_famous_mofs_rewritten = rename_zeopp_df(df_zeopp_famous_mofs)
df_zeopp_zeolites_rewritten = rename_zeopp_df(df_zeopp_zeolites)

## 3. Before merging, let's add the group column

In [36]:
import os
sampled = os.listdir('structures_for_study')

In [37]:
sampled_names = [n.strip('.cif') for n in sampled]

In [32]:
df_zeopp_cofs_rewritten['group'] = ['COFs'] * len(df_zeopp_cofs_rewritten)
df_zeopp_mofs_rewritten['group'] = ['MOFs'] * len(df_zeopp_mofs_rewritten)
df_zeopp_famous_mofs_rewritten['group'] = ['MOFs'] * len(df_zeopp_famous_mofs_rewritten)
df_zeopp_zeolites_rewritten['group'] = ['zeolites'] * len(df_zeopp_zeolites_rewritten)

In [33]:
df_zeopp_concat = pd.concat([df_zeopp_cofs_rewritten, 
                             df_zeopp_mofs_rewritten, 
                             df_zeopp_famous_mofs_rewritten,
                             df_zeopp_zeolites_rewritten])

In [34]:
df_zeopp_concat.head()

largest_free_sphere  accessible_surface_area   density     poav  \
0             16.03454                  7267.46  0.173311  0.89428   
1              8.62197                  5173.00  0.394807  0.75486   
2             17.28794                  1581.94  0.725925  0.61143   
3              8.10818                  1655.89  0.730299  0.39137   
4             22.10199                  6289.70  0.146333  0.92453   

   largest_included_free_sphere  pocket_surface_area  largest_included_sphere  \
0                      22.72268                  0.0                 22.72268   
1                      13.26023                  0.0                 13.26023   
2                      17.62620                  0.0                 17.62620   
3                       8.32513                  0.0                  8.32513   
4                      31.72634                  0.0                 31.72634   

      name group  
0  16110N3  COFs  
1  14050N3  COFs  
2  18111N2  COFs  
3  17090N3  COFs  
4  16130N3  COFs

In [38]:
new_group_column = []
for n in df_zeopp_concat['name'].values:
    print(n)
    if n in sampled_names:
        new_group_column.append(True)
    else: 
        new_group_column.append(False)

16110N3
14050N3
18111N2
17090N3
16130N3
13012N2
14031N2
15183N2
13051N2
13073N2
15180N2
15181N2
07001N2
15211N2
18022N3
16411C2
16412C2
12001N2
16031N2
13122N2
10000N2
16381N2
16210N2
16112N3
16020N2
15130N2
17130N2
18134N3
17000N2
13080N2
14100N2
11001N2
12030N2
16060N2
16311N2
14071N2
08010N2
12023N2
16390N2
12011N2
15080N2
17153N2
17163N3
16330N2
15120N2
17030N2
15182N2
18082N2
16220N2
15171N2
17120N2
10001N2
16420N2
15160N2
18122N3
15190N2
16061N2
18112N2
18030N2
18010N3
13161N2
16030N2
13010N2
16370N2
16331N2
17162N3
13160N2
13142N2
12062N2
14070N2
18083N2
14090N2
18100N2
13050N2
18132N3
18000C3
15200N3
13121N2
16300N2
11030N2
13141N2
12000N2
17091N3
16100N2
18081N2
15110N2
17160N3
16280C2
18021N3
18131N3
18080N2
15070N2
16340N2
16180C2
13077N2
15021N2
17170N2
16450N2
16140N2
08012N2
16040N2
18060N2
17191N2
12061N2
13110N2
16174N2
13100N2
16011N2
11003N2
13090N2
16051N2
17080C2
16032N2
17131N2
16371N2
14060N2
15000N2
15072N2
16251N3
18090N2
14091N2
18110N2
16070N2
17020N2
16320N2


In [39]:
df_zeopp_concat['sampled'] = new_group_column

In [40]:
merged_raspa_zeopp_df = pd.merge(df_raspa_merged, df_zeopp_concat, on='name', how='right')

In [41]:
merged_raspa_zeopp_df[merged_raspa_zeopp_df['sampled'] == True]

name   kh_12_no_tc  kh_error_12_no_tc  \
0                 13030N2  2.056090e-56       1.612610e-56   
1                 15030N2  2.701440e-06       1.364100e-08   
2                 16411C2  3.104940e-06       5.944330e-08   
3                 17030N2  3.268900e-06       1.313540e-08   
4                 17040N2  3.848890e-06       2.381050e-08   
5                 17120N2  2.587110e-05       3.291490e-07   
6                 17163N3  7.339970e-06       6.980640e-08   
7                     AFT  4.176940e-04       1.267960e-05   
8        BENXUP_clean_min  3.565590e-06       9.019850e-08   
9                  Cu-BTC  1.541540e-05       3.347720e-07   
10    DIYTEM_freeONLY_min  1.047070e-05       7.597320e-08   
11                    GOO  6.103350e-04       1.267830e-05   
12       IBICIH_clean_min  3.057100e-06       1.191550e-08   
13                IRMOF-1  4.495270e-06       1.638430e-08   
14               IRMOF-10  5.230540e-06       2.076780e-08   
15                    MTT  3.761370e-04       3.573800e-06   
16              Mg-MOF-74  9.581460e-06       7.172400e-08   
17       NOCKUM_clean_min  1.093820e-05       1.092710e-07   
18       NUTQEZ_clean_min  7.726890e-06       2.731940e-08   
19       PIHNUQ_clean_min  1.508880e-05       2.780900e-07   
20       RIVDEF_clean_min  3.546580e-06       5.124800e-08   
21                SBMOF-1  1.051940e-04       2.933550e-06   
22                    SFN  4.796110e-05       6.652550e-07   
23                    SFS  4.231910e-04       1.027780e-05   
24                    TSC  3.247300e-03       1.395130e-04   
25    TUDJIM_freeONLY_min  4.847800e-06       4.141540e-08   
26                    UEI  8.835480e-04       3.001650e-05   
27                 UIO-66  1.347610e-05       1.777500e-07   
28                 UMCM-1  5.257370e-06       3.850220e-08   
29     XOJWID_charged_min  8.123320e-06       1.319130e-07   
30       YUJNIB_clean_min  1.371690e-06       3.597890e-08   
31                  ZIF-4  3.325830e-05       9.098160e-07   
32                  ZIF-8  5.580410e-06       5.724450e-08   
33                15072N2  5.131760e-06       8.164170e-08   
34                18091N2  8.023260e-06       8.147890e-08   
35                    EMT  3.274750e-05       7.054300e-06   
173               16083N2           NaN                NaN   
1173              MIL-125           NaN                NaN   
1327                  MFI           NaN                NaN   
1345                  PAU           NaN                NaN   

      loading_65_12_no_tc  loading_5_8_12_no_tc  loading_35_12_no_tc  \
0                0.000000              0.000000             0.000000   
1              128.065247             19.244052            87.614821   
2              110.779208             26.670975            85.559362   
3              117.527150             16.482524            77.417326   
4              144.987710             32.470629           115.025790   
5              118.188049             64.652008           108.517804   
6              177.884660             26.667428           121.057725   
7              227.839619            195.944468           223.988217   
8               89.670889             42.710813            84.166516   
9              225.138132             67.617816           187.303900   
10             117.420682             69.060449           108.414570   
11              88.214675             87.765156            88.152349   
12             122.990640             37.303740           105.199006   
13             212.391750             33.705041           155.594448   
14             177.652078             21.389832           112.722099   
15             131.697328             89.787407           121.928617   
16             230.492115             90.616434           203.521762   
17             185.326476             74.722399           159.720380   
18             216.575316             58.282021           179.102872   
19              52.826120             

### Add the principal components 

In [42]:
import pandas as pd
pca_df = pd.read_csv('dimensionality_reduced.csv')

In [43]:
pca_df.head()

name     pca_1     pca_2     mds_1     mds_2
0  16110N3  5.343021  0.595102 -5.813140  1.086589
1  14050N3  2.976708 -0.122522 -3.802906 -0.157914
2  18111N2  2.667902  0.054064 -2.519662  0.847286
3  17090N3  1.119475 -1.198866 -1.854988 -1.738373
4  16130N3  6.474157  1.290515 -6.431392  2.474329

In [44]:
pca_df.describe()

pca_1         pca_2        mds_1         mds_2
count  1.375000e+03  1.375000e+03  1375.000000  1.375000e+03
mean   8.268134e-17 -4.134067e-17     0.000000 -4.134067e-17
std    2.264134e+00  1.431615e+00     2.359904  1.802319e+00
min   -4.248412e+00 -2.305243e+00    -6.843412 -5.100968e+00
25%   -2.052200e+00 -8.227152e-01    -1.565472 -1.053433e+00
50%   -4.321461e-01 -3.479479e-01     0.132267 -3.043460e-01
75%    1.374015e+00  4.866133e-01     2.229651  7.042211e-01
max    7.527838e+00  1.090775e+01     6.858094  1.463943e+01

In [45]:
merged_raspa_zeopp_df.head()

name   kh_12_no_tc  kh_error_12_no_tc  loading_65_12_no_tc  \
0  13030N2  2.056090e-56       1.612610e-56             0.000000   
1  15030N2  2.701440e-06       1.364100e-08           128.065247   
2  16411C2  3.104940e-06       5.944330e-08           110.779208   
3  17030N2  3.268900e-06       1.313540e-08           117.527150   
4  17040N2  3.848890e-06       2.381050e-08           144.987710   

   loading_5_8_12_no_tc  loading_35_12_no_tc  loading_65_error_12_no_tc  \
0              0.000000             0.000000                   0.000000   
1             19.244052            87.614821                   1.721082   
2             26.670975            85.559362                   1.801933   
3             16.482524            77.417326                   0.646303   
4             32.470629           115.025790                   1.280339   

   loading_5_8_error_12_no_tc  loading_35_error_12_no_tc  dc_12_no_tc  ...  \
0                    0.000000                   0.000000     0.000000  ...   
1                    0.471007                   1.910243   108.821195  ...   
2                    0.908552                   1.466130    84.108233  ...   
3                    0.707367                   1.430119   101.044625  ...   
4                    1.134927                   1.713823   112.517081  ...   

     dc_20_tc  largest_free_sphere  accessible_surface_area   density  \
0    0.000000              1.30303                     0.00  1.669780   
1  124.971520             18.38514                  1910.89  0.581115   
2   91.431330             10.03477                  1502.00  0.849852   
3  114.511976             31.61204                  2131.77  0.413930   
4  118.457958             12.51715                  1579.13  0.736546   

      poav  largest_included_free_sphere  pocket_surface_area  \
0  0.00000                       2.61808                  0.0   
1  0.60729                      18.68522                  0.0   
2  0.39497                      10.55425                  0.0   
3  0.72619                      31.79267                  0.0   
4  0.49761                      12.95632                  0.0   

   largest_included_sphere  group  sampled  
0                  2.61808   COFs     True  
1                 18.68522   COFs     True  
2                 10.55425   COFs     True  
3                 31.79267   COFs     True  
4                 12.95632   COFs     True  

[5 rows x 46 columns]

In [46]:
merged_4_materials_cloud = pca_df.merge(merged_raspa_zeopp_df, left_on='name', right_on='name')

In [47]:
merged_4_materials_cloud.to_csv('properties.csv', index=False)

## 4. Now, copy over the RDFs

In [48]:
from glob import glob
from pathlib import Path

rdfs_a = glob('/home/kevin/Documents/uni/EPFL/master_thesis/tailcorrection/rdf_without_blocked_pockets_correct_unit_cell/*/RDF_Framework_CH4_sp3.dat')
rdfs_b = glob('/home/kevin/Documents/uni/EPFL/master_thesis/tailcorrection/rdf_blocked_pockets/*/RDF_Framework_CH4_sp3.dat')

In [52]:
rdf_list = []

for rdf in rdfs_a:
    p = Path(rdf).parts[-2]
    if p not in ['EMT', '15072N2', '16083N2', '18091N2']:
        rdf_list.append(rdf)

for rdf in rdfs_b:
    p = Path(rdf).parts[-2]
    if p in ['EMT', '15072N2', '16083N2', '18091N2']:
        rdf_list.append(rdf)

In [53]:
len(rdf_list)

39

In [55]:
import shutil

for rdf in rdf_list:
    name = Path(rdf).parts[-2]
    shutil.copyfile(rdf, 
                    os.path.join('/home/kevin/Documents/uni/EPFL/master_thesis/tailcorrection/materials_cloud_entry/structure-property-visualizer/data/rdfs', name + '.csv'))

## 5. Get boundaries for plots

In [56]:
merged_raspa_zeopp_df.describe()

kh_12_no_tc  kh_error_12_no_tc  loading_65_12_no_tc  \
count    36.000000       3.600000e+01            36.000000   
mean      0.000175       6.231623e-06           143.664207   
std       0.000563       2.373596e-05            54.737188   
min       0.000001       1.139300e-08            52.826120   
25%       0.000004       3.759155e-08           110.944382   
50%       0.000008       8.581420e-08           136.290982   
75%       0.000025       4.438815e-07           179.632377   
max       0.003247       1.395130e-04           257.295301   

       loading_5_8_12_no_tc  loading_35_12_no_tc  loading_65_error_12_no_tc  \
count             36.000000            36.000000                  36.000000   
mean              57.472453           118.641955                   1.388128   
std               42.827309            47.491028                   0.963474   
min               16.425522            47.787786                   0.025606   
25%               27.231831            86.141638                   0.697834   
50%               44.501200           111.296826                   1.199721   
75%               67.978474           135.889212                   1.872028   
max              195.794291           240.466790                   4.778873   

       loading_5_8_error_12_no_tc  loading_35_error_12_no_tc  dc_12_no_tc  \
count                   36.000000                  36.000000    36.000000   
mean                     1.129562                   1.338495    86.191754   
std                      0.578761                   0.824222    53.631718   
min                      0.122337                   0.034325     0.244756   
25%                      0.700476                   0.824551    41.699515   
50%                      1.102024                   1.197345    84.925645   
75%                      1.512334                   1.681935   124.418752   
max                      2.433043                   3.351692   178.686710   

        kh_12_tc  ...  loading_5_8_error_20_tc  loading_35_error_20_tc  \
count  36.000000  ...                36.000000               36.000000   
mean    0.000241  ...                 0.572962                0.823497   
std     0.000742  ...                 0.439058                0.609905   
min     0.000002  ...                 0.073030                0.018350   
25%     0.000005  ...                 0.296981                0.474775   
50%     0.000009  ...                 0.520644                0.753930   
75%     0.000029  ...                 0.737153                0.971407   
max     0.004211  ...                 2.221110                2.854526   

         dc_20_tc  largest_free_sphere  accessible_surface_area      density  \
count   36.000000          1378.000000              1378.000000  1378.000000   
mean    90.597464             7.877498              1153.717350     1.173383   
std     58.346384             7.399244              1361.760349     0.502326   
min      0.168286             0.836270                 0.000000     0.116104   
25%     40.291801             3.422630                 0.000000     0.766061   
50%     92.052063             4.902370               736.131500     1.168430   
75%    134.434176             8.101560              1781.462500     1.569680   
max    191.315574            44.502790              8560.100000     3.162480   

              poav  largest_included_free_sphere  pocket_surface_area  \
count  1378.000000                   1378.000000          1378.000000   
mean      0.327981                      9.540151            14.525968   
std       0.275943                      7.179141            47.667833   
min       0.000000                      2.618080             0.000000   
25%       0.000000                      4.936325             0.000000   
50%       0.340645                      6.674990             0.000000   
75%       0.563247                     11.456800             3.760692   
max       0.932720                     44.620260           586.045000 

AttributeError: 'module' object has no attribute 'resources'